In [1]:
import numpy as np
import pickle
import pandas as pd

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
with open('../data/HCD35_pos.pickle', mode='rb') as fp:
    df_3 = pickle.load(fp)
    
df = pd.concat([
    df_3,
    pd.read_csv('../data/decomp_pos.csv').drop('Subclass', axis=1)
], axis=1)

In [3]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [4]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

pickle.dump(clf, open('../model/rf_pos_HCD35+decomp.sav', 'wb'))

In [5]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

pickle.dump(gbm, open('../model/lgbm_pos_HCD35+decomp.sav', 'wb'))

[1]	valid_0's multi_logloss: 1.28903
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.20755
[3]	valid_0's multi_logloss: 1.14199
[4]	valid_0's multi_logloss: 1.08496
[5]	valid_0's multi_logloss: 1.03621
[6]	valid_0's multi_logloss: 0.991759
[7]	valid_0's multi_logloss: 0.950848
[8]	valid_0's multi_logloss: 0.913097
[9]	valid_0's multi_logloss: 0.880727
[10]	valid_0's multi_logloss: 0.851439
[11]	valid_0's multi_logloss: 0.823603
[12]	valid_0's multi_logloss: 0.799053
[13]	valid_0's multi_logloss: 0.776538
[14]	valid_0's multi_logloss: 0.756655
[15]	valid_0's multi_logloss: 0.735506
[16]	valid_0's multi_logloss: 0.715997
[17]	valid_0's multi_logloss: 0.697984
[18]	valid_0's multi_logloss: 0.679301
[19]	valid_0's multi_logloss: 0.664049
[20]	valid_0's multi_logloss: 0.650217
[21]	valid_0's multi_logloss: 0.634705
[22]	valid_0's multi_logloss: 0.619892
[23]	valid_0's multi_logloss: 0.608396
[24]	valid_0's multi_logloss: 0.596699
[25]	valid_0's mu

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        device='gpu', importance_type='split', learning_rate=0.1,
        max_depth=-1, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
        objective='multiclass', random_state=None, reg_alpha=0.0,
        reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)

In [6]:
# set parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 0
params['updater'] = 'grow_gpu_hist'

In [8]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

pickle.dump(model, open('../model/xgb_pos_HCD35+decomp.sav', 'wb'))

In [9]:
with open('../data/HCD35_neg.pickle', mode='rb') as fp:
    df_3 = pickle.load(fp)
    
df = pd.concat([
    df_3,
    pd.read_csv('../data/decomp_neg.csv').drop('Subclass', axis=1)
], axis=1)

In [10]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [11]:
# define and fit 
clf = rf()
clf = clf.fit(X_train, y_train)

pickle.dump(clf, open('../model/rf_ng_HCD35+decomp.sav', 'wb'))

In [12]:
# define and fit
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

pickle.dump(gbm, open('../model/lgbm_ng_HCD35+decomp.sav', 'wb'))

In [13]:
# setting parameters
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [14]:
# define and fit 
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

pickle.dump(model, open('../model/xgb_ng_HCD35+decomp.sav', 'wb'))